![NYC Skyline](img/nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many [Airbnb](https://www.airbnb.com/) listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this notebook, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx` (Excel files).

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**datasets/airbnb_price.csv**
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**datasets/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**datasets/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed


In [48]:
# We've loaded your first few packages for you in the first cell. Please feel free to add as many cells as you like!
suppressMessages(library(dplyr)) # This line is required to check your answer correctly
options(readr.show_types = FALSE) # This line is required to check your answer correctly
library(readr)
library(readxl)
library(stringr)

airbnb_price = read_csv('data/airbnb_price.csv')
airbnb_room_type = read_excel('data/airbnb_room_type.xlsx')
airbnb_last_review = read_tsv('data/airbnb_last_review.tsv')

Rows: 25209 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): price, nbhood_full
dbl (1): listing_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 25209 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): host_name, last_review
dbl (1): listing_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [49]:
airbnb = airbnb_price %>% inner_join(airbnb_room_type, by = 'listing_id') %>% 
	inner_join(airbnb_last_review,by='listing_id')
str(airbnb)
unique(airbnb$room_type)

#cleaning dataset
#change type of last_review and price
#change airbnb$room_type to lower_case

# Question 1: What is the average listing price? 
avg_price = airbnb %>%
  mutate(price = str_remove(price, ' dollars')) %>%
  mutate(price = as.numeric(price)) %>%
  summarize(avg_price = mean(price)) %>% as.numeric(avg_price)
avg_price

# Question 2: How many of the listings are private rooms? 
nb_private = airbnb %>% mutate(room_type = str_to_lower(room_type)) %>%
		count(room_type) %>% filter(room_type == 'private room')
nb_private
nb_private_rooms = nb_private$n

#Question 3: Which listing was most recently reviewed? 
#https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/strptime
date = airbnb %>%
  mutate(date = as.Date(last_review, format = '%B %d %Y')) %>% 
summarize(first_review =min(date) , last_review=max(date))
date
first_reviewed = date$first_review
last_reviewed = date$last_review
first_reviewed 
last_reviewed

#final table
review_dates = data.frame(first_reviewed, last_reviewed, nb_private_rooms, avg_price)
review_dates

spc_tbl_ [25,209 × 7] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ listing_id : num [1:25209] 2595 3831 5099 5178 5238 ...
 $ price      : chr [1:25209] "225 dollars" "89 dollars" "200 dollars" "79 dollars" ...
 $ nbhood_full: chr [1:25209] "Manhattan, Midtown" "Brooklyn, Clinton Hill" "Manhattan, Murray Hill" "Manhattan, Hell's Kitchen" ...
 $ description: chr [1:25209] "Skylit Midtown Castle" "Cozy Entire Floor of Brownstone" "Large Cozy 1 BR Apartment In Midtown East" "Large Furnished Room Near B'way" ...
 $ room_type  : chr [1:25209] "Entire home/apt" "Entire home/apt" "Entire home/apt" "private room" ...
 $ host_name  : chr [1:25209] "Jennifer" "LisaRoxanne" "Chris" "Shunichi" ...
 $ last_review: chr [1:25209] "May 21 2019" "July 05 2019" "June 22 2019" "June 24 2019" ...
 - attr(*, "spec")=
  .. cols(
  ..   listing_id = col_double(),
  ..   price = col_character(),
  ..   nbhood_full = col_character()
  .. )
 - attr(*, "problems")=<externalptr> 


[1] "Entire home/apt" "private room"    "Private room"    "entire home/apt"
[5] "PRIVATE ROOM"    "shared room"     "ENTIRE HOME/APT" "Shared room"    
[9] "SHARED ROOM"

[1] 141.7779

room_type,n
<chr>,<int>
private room,11356


first_review,last_review
<date>,<date>
2019-01-01,2019-07-09


[1] "2019-01-01"

[1] "2019-07-09"

first_reviewed,last_reviewed,nb_private_rooms,avg_price
<date>,<date>,<int>,<dbl>
2019-01-01,2019-07-09,11356,141.7779
